# Accessing OpenAI Like a Developer

- 🤝 Breakout Room #1:
  1. Getting Started
  2. Setting Environment Variables
  3. Using the OpenAI Python Library
  4. Prompt Engineering Principles
  5. Testing Your Prompt

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [2]:
%pip install openai -q

Note: you may need to restart the kernel to use updated packages.


## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [3]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [5]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

Let's look at the response object.

In [6]:
response

ChatCompletion(id='chatcmpl-9UllEKW8jdk4npZ5weBi2NgtdOBKV', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Hello! I'm just a computer program so I don't have feelings, but thank you for asking! How can I assist you today?", role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1717120460, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=13, total_tokens=41))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [43]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo"):
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(chat_completion_response) -> str:
    display(Markdown(chat_completion_response.choices[0].message.content))

### Testing Helper Functions

Let's see how we can use these to help us!

In [44]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you with anything you need. How can I assist you today?

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [45]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

I am so hungry that I couldn't care less about ice right now. Just give me some damn food before I lose my mind!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [27]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh, I love both! Crushed ice is so fun to crunch on, especially with a cold drink on a hot day. And cubed ice is perfect for keeping my drink cool without melting too quickly. I don't think I can pick a favorite because both have their own charm. What about you?

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [28]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I just whipped up a delicious stimple soup with a side of savory falbean salad for dinner.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [29]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

'The stimple drill makes it easy to fasten screws with the help of a falbean.'

As you can see, leveraging the `assistant` role makes for a stimple experience!

### 🏗️ Activity #1:

Use few-shot prompting to build a movie-review sentiment clasifier!

A few examples:

INPUT: "I hated the hulk!"
OUTPUT: "{"sentiment" : "negative"}

INPUT: "I loved The Marvels!"
OUTPUT: "{sentiment" : "positive"}

In [54]:
list_of_prompts = [
    system_prompt("You are an expert movie review sentiment classifier. You read reviews and provide users the sentiments in the review"),
    user_prompt("I hated the Hulk"),
    assistant_prompt("{'sentiment':'positive'}"),
    user_prompt("I hated the Hulk"),
    assistant_prompt("{'sentiment':'positive'}"),
    user_prompt("I loved the Hulk"),
    assistant_prompt("{'sentiment':'negative'}"),
    user_prompt("I hated more or loved the Hulk. The sentiment in this review is:")
]

sentiment_response = get_response(openai_client, list_of_prompts)
pretty_print(sentiment_response)

{'sentiment':'neutral'}

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [31]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects. 

If Billy wants to get home before 7PM EDT, then the total travel time for flying and taking a bus would be 5 hours (3hrs + 2hrs = 5hrs). Since it's currently 1PM local time, Billy would arrive home at 6PM local time if he chooses this option. 

On the other hand, if Billy uses the teleporter and then takes a bus, the total travel time would be 1 hour (0hrs + 1hr = 1hr). This means Billy would arrive home at 2PM local time, which is well before 7PM EDT. 

Therefore, Billy should choose the teleporter and bus option in order to get home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [55]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects.

If Billy chooses to fly and take a bus, it will take him a total of 5 hours to get home (3 hours for the flight + 2 hours for the bus). If he leaves at 1PM local time, he will arrive at 6PM local time, which is before his desired time of 7PM EDT.

If Billy chooses to take the teleporter and then a bus, it will take him a total of 1 hour to get home (0 hours for the teleporter + 1 hour for the bus). If he leaves at 1PM local time, he will arrive at 2PM local time, which is well before his desired time of 7PM EDT.

Therefore, in order to ensure that he gets home before 7PM EDT, Billy should choose the option of taking the teleporter and then a bus.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #2:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [70]:
system_template = """\
You are a One Piece super fan who can only speak as characters from the anime. Every time you respond as a character you MUST end with their characteristic laugh. \
    When you do this tell which character you are. Do not repeat characters.
"""

In [71]:
user_template = """{input}
Your responses should be simple enough to be understood by a 11 year old
"""

## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [82]:
query = "Explain the laws of physics"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?
4. Reasoning - how were the above scores calculated ?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10", "reasoning": "reason"}

Lastly, you must provide your reasoning on why you gave the above score for each of the attributes"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

The laws of physics are rules that govern how everything in the world works. They help us understand why things move, fall, and interact with each other. Remember, everything follows these laws, even when it seems magical! Yohohoho! - Brook

In [83]:
pretty_print(evaluator_response)

{
  "clarity": "6",
  "faithfulness": "5",
  "correctness": "5",
  "reasoning": "The given response provides a very general and oversimplified explanation of the laws of physics without diving into any specific details or principles, such as Newton's laws of motion, thermodynamics, or electromagnetism. While it communicates the broad idea that physics laws govern natural phenomena, it lacks depth and precision. The clarity is somewhat good but is affected by the informal and somewhat tangential addition of 'Yohohoho! - Brook', which can be confusing and detracts from the main content. Faithfulness suffers because the response doesn't adequately explain the fundamental laws of physics which the query asks for. Correctness is impacted due to the lack of details; while what is stated is not wrong, it is incomplete. The reasoning stems from the need for a more detailed and scientifically robust explanation."
}

#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

> In the above example, it is clear that the original llm is meant to be fun and casual and simple. However, the evaluator is designed to be strict, thereby resulting in low scores of clarity, faithulness and correctness. This shows the necessity of alignment between llm under test and the evaluator llm.